# Signal

In this notebook we measure the signal quality of the signal source and signal source with amplifier.

In [13]:
import os
import control
import numpy as np
import pandas as pd

from time import sleep
from matplotlib import pyplot as plt

scope = control.MSOX6004A('172.22.22.30')
scope.holla()

The oscilloscope is not able to resolve the sine shape at sweep duration in the millisecond range which makes any frequency analysis very hard, thus we reduce the sweep duration to microseconds. The resolution is then high enough to clearly distinguish the period but still not too far to our target duration of 26.84ms.

In [33]:
control.aod_h(frequency=[90e6, 110e6], duration=10e-6, amplitude=np.ones(1024), nodwells=[False, False])

scope.single()

sleep(.5)

control.trigger()

t_h, U_h = scope.capture(1)

We save the data to a compressed HDF5 file for faster io and reduced disk footprint compared to CSV.

In [40]:
os.makedirs('../dataset/signal', exist_ok=True)

signal_h = pd.DataFrame({ 'time': t_h, 'voltage': U_h })
signal_h.to_hdf('../dataset/signal/source.h5', 'horizontal', complevel=9)

Now we need to change the signal source to measure the other DDS output.

In [34]:
control.aod_v(frequency=[90e6, 110e6], duration=10e-6, amplitude=np.ones(1024), nodwells=[False, False])

scope.single()

sleep(.5)

control.trigger()

t_v, U_v = scope.capture(1)

In [42]:
signal_v = pd.DataFrame({ 'time': t_v, 'voltage': U_v })
signal_v.to_hdf('../dataset/signal/source.h5', 'vertical', complevel=9)